In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../DataSet_DummiesWT.csv")

In [3]:
df.head()

,character,max_hp,entering_hp,damage_taken,enemies,Adaptation,Adaptation+1,Alpha,Alpha+1,Apotheosis,...,Unceasing Top,Vajra,Velvet Choker,VioletLotus,WarpedTongs,Whetstone,White Beast Statue,WingedGreaves,Yang,level_asc_floor
0,WATCHER,72,72,0,39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,WATCHER,72,72,0,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,WATCHER,72,72,15,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,WATCHER,72,57,17,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
4,WATCHER,72,61,0,34,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,19


In [4]:
df.level_asc_floor.value_counts()

level_asc_floor
7       346
8       310
3       307
5       269
25      266
       ... 
1750      1
980       1
1778      1
1766      1
1889      1
Name: count, Length: 813, dtype: int64

In [5]:
df.drop(columns=["character"], inplace=True)

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

# Separar características y etiqueta
X = df.drop(columns=["level_asc_floor"])  
y = df.drop(columns=["level_asc_floor"])   

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar los datos usando MaxAbsScaler
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificar tamaños de los conjuntos de entrenamiento y prueba
print(f"Tamaño de X_train_scaled: {X_train_scaled.shape}")
print(f"Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de X_test_scaled: {X_test_scaled.shape}")
print(f"Tamaño de y_test: {y_test.shape}")

# # Guardar los datos escalados para usarlos más tarde
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
# X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)
# X_train_scaled_df.to_csv('X_train_scaled.csv', index=False)
# X_test_scaled_df.to_csv('X_test_scaled.csv', index=False)
# pd.DataFrame(y_train).to_csv('y_train.csv', index=False)
# pd.DataFrame(y_test).to_csv('y_test.csv', index=False)


Tamaño de X_train_scaled: (12729, 386)
Tamaño de y_train: (12729, 386)
Tamaño de X_test_scaled: (3183, 386)
Tamaño de y_test: (3183, 386)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError

# Construir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model.compile(optimizer=RMSprop(),
              loss='mse',
              metrics=[MeanAbsoluteError(), MeanSquaredError()])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, 
                    epochs=100, 
                    validation_split=0.2, 
                    batch_size=32, 
                    verbose=1)

# Evaluar el modelo
evaluation = model.evaluate(X_test_scaled, y_test, verbose=1)

# Resultados
print("Evaluación del modelo:")
print(f"MAE: {evaluation[1]}")
print(f"MSE: {evaluation[2]}")

evaluation

Epoch 1/100
319/319 [==============================] - 4s 6ms/step - loss: 25.6295 - mean_absolute_error: 0.9135 - mean_squared_error: 25.6295 - val_loss: 25.7333 - val_mean_absolute_error: 0.9033 - val_mean_squared_error: 25.7333
Epoch 2/100
319/319 [==============================] - 1s 5ms/step - loss: 25.6260 - mean_absolute_error: 0.9171 - mean_squared_error: 25.6260 - val_loss: 25.7338 - val_mean_absolute_error: 0.9489 - val_mean_squared_error: 25.7338
Epoch 3/100
319/319 [==============================] - 2s 5ms/step - loss: 25.6257 - mean_absolute_error: 0.9172 - mean_squared_error: 25.6257 - val_loss: 25.7324 - val_mean_absolute_error: 0.9158 - val_mean_squared_error: 25.7324
Epoch 4/100
319/319 [==============================] - 2s 5ms/step - loss: 25.6255 - mean_absolute_error: 0.9172 - mean_squared_error: 25.6255 - val_loss: 25.7325 - val_mean_absolute_error: 0.9082 - val_mean_squared_error: 25.7325
Epoch 5/100
319/319 [==============================] - 2s 5ms/step - loss: 2

[25.816429138183594, 0.9262692332267761, 25.816429138183594]

In [8]:
y.describe()

,max_hp,entering_hp,damage_taken,enemies,Adaptation,Adaptation+1,Alpha,Alpha+1,Apotheosis,Apotheosis+1,...,Turnip,Unceasing Top,Vajra,Velvet Choker,VioletLotus,WarpedTongs,Whetstone,White Beast Statue,WingedGreaves,Yang
count,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,...,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000,15912.000000
mean,73.264203,51.864442,8.554739,31.712104,0.049460,0.017660,0.019419,0.019168,0.003205,0.006913,...,0.024384,0.021430,0.043175,0.012883,0.043992,0.018602,0.000691,0.029098,0.013323,0.026018
std,6.871744,18.111007,12.308497,17.353367,0.225639,0.146189,0.137998,0.137119,0.056525,0.082859,...,0.154243,0.144819,0.203257,0.112775,0.205084,0.135120,0.026284,0.168085,0.114659,0.159194
min,36.000000,1.000000,-39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,72.000000,40.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,72.000000,54.000000,4.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,77.000000,65.000000,14.000000,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,123.000000,123.000000,78.000000,63.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError

# Separar características y etiqueta
X = df.drop(columns=["level_asc_floor"])  
y = df["level_asc_floor"]   

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar los datos usando MaxAbsScaler
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificar tamaños de los conjuntos de entrenamiento y prueba
print(f"Tamaño de X_train_scaled: {X_train_scaled.shape}")
print(f"Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de X_test_scaled: {X_test_scaled.shape}")
print(f"Tamaño de y_test: {y_test.shape}")

# Construir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)

    
])

# Compilar el modelo
model.compile(optimizer=RMSprop(),
              loss='mse',
              metrics=[MeanAbsoluteError(), MeanSquaredError()])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, 
                    epochs=100, 
                    validation_split=0.2, 
                    batch_size=32, 
                    verbose=1)

# Evaluar el modelo
evaluation = model.evaluate(X_test_scaled, y_test, verbose=1)

# Resultados
print("Evaluación del modelo:")
print(f"MAE: {evaluation[1]}")
print(f"MSE: {evaluation[2]}")

print(evaluation)

Tamaño de X_train_scaled: (12729, 386)
Tamaño de y_train: (12729,)
Tamaño de X_test_scaled: (3183, 386)
Tamaño de y_test: (3183,)
Epoch 1/100
319/319 [==============================] - 3s 6ms/step - loss: 644296.4375 - mean_absolute_error: 571.1265 - mean_squared_error: 644296.4375 - val_loss: 421938.7188 - val_mean_absolute_error: 538.4620 - val_mean_squared_error: 421938.7188
Epoch 2/100
319/319 [==============================] - 1s 4ms/step - loss: 380373.3125 - mean_absolute_error: 520.7958 - mean_squared_error: 380373.3125 - val_loss: 331480.5625 - val_mean_absolute_error: 491.8676 - val_mean_squared_error: 331480.5625
Epoch 3/100
319/319 [==============================] - 1s 4ms/step - loss: 259097.9688 - mean_absolute_error: 427.2803 - mean_squared_error: 259097.9688 - val_loss: 182196.4531 - val_mean_absolute_error: 347.9726 - val_mean_squared_error: 182196.4531
Epoch 4/100
319/319 [==============================] - 1s 4ms/step - loss: 121551.4453 - mean_absolute_error: 283.714

[28596.69921875, 107.35449981689453, 28596.69921875]

In [13]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test_scaled)

# Calcular el R2
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2}")

evaluation

100/100 [==============================] - 2s 3ms/step
R^2: 0.935797377395581


[28596.69921875, 107.35449981689453, 28596.69921875]

In [21]:
df.level_asc_floor.max()

2100

MAE (95.89): Indica que el error promedio de las predicciones es 95.89 unidades, lo cual es bajo comparado con el rango de la variable (1 a 2100). Esto sugiere que las predicciones son bastante precisas.

MSE (22985): El error cuadrático medio, que penaliza más los errores grandes, es relativamente bajo considerando el rango de la variable. La raíz cuadrada del MSE (aproximadamente 151.62) también indica un error bajo.

R^2 (0.9358): Muestra que el modelo explica el 93.58% de la varianza en la variable objetivo, lo cual es excelente.

Conclusión: Tiene errores bajos (MAE y MSE) y explica la mayor parte de la variabilidad en los datos (R^2 alto).